In [2]:
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
import codecs
import string
from fuzzy_match import match
from fuzzy_match import algorithims
import time
import io
import time
import numpy as np
import string

# Scrap one page and bluid df

In [3]:
# scrap one page
#url = f'http://www.moto-selection.com/moto-occasion/page-1.html'
#response = requests.get(url)
#file_name = 'page_scraped'

#with open(f"{file_name}.html", "w")  as file:
            #file.write(response.text)
            #file.close()

In [5]:
# build to dataframe
f=codecs.open('page_scraped.html')
soup = BeautifulSoup(f, "html.parser")

data = {'uniq_id':[],
        'brand':[],
        'model':[],
        'price':[],
        'mileage':[],
        'bike_year':[]}

for bike in soup.find_all('div', class_="announces_list_item")[0:1]:
    url_bike = bike.find("a", class_="title_link item_link").get('href')
    uniq_id = url_bike.split('/')[-1].replace('.html', '')
    brand = bike.find("a", class_="title_link item_link").find('span', itemprop="brand").text.lower()
    brand = brand.replace(r"\(.*\)","").replace('[^\w\s]','').replace(" ","")
    model = bike.find("a", class_="title_link item_link").find('span', itemprop="name").text.lower()
    model = model.replace(r"\(.*\)","").replace('[^\w\s]','').replace(" ","")
    price = int(bike.find('div', class_="item_price").find('span', itemprop="price").text.replace(" ",''))
    mileage = int(bike.find('div', class_="item_desc").find(class_="item_desc_km").text.replace(" ",'').replace("Km",""))
    bike_year = int(bike.find('div', class_="item_desc").find(class_="item_desc_millesime").text.replace('Année ', ''))
                                                        
    data['uniq_id'].append(uniq_id)
    data['brand'].append(brand)
    data['model'].append(model)
    data['mileage'].append(mileage)
    data['bike_year'].append(bike_year)
    data['price'].append(price)
    
master_data = pd.DataFrame(data)

#remove punctuation
def remove_punctuations(text):
    for punctuation in string.punctuation:
      text = text.replace(punctuation, '')
    return text

#remove punctuation
master_data.brand = master_data.brand.apply(remove_punctuations)
master_data.model = master_data.model.apply(remove_punctuations)

master_data

,uniq_id,brand,model,price,mileage,bike_year
0,1189664,yamaha,niken,10590,4100,2018


# FUZZY MATCH

In [6]:
#import motorcycle databse
motorcycle_database = pd.read_csv('motorcycle_database/ebay.csv')
motorcycle_database = motorcycle_database[(motorcycle_database.type_db == "motorcycle") | (motorcycle_database.type_db == "scooter")]

def match_brand(choices, to_match, match_type='levenshtein'): #match_type options include trigram, cosine, levenshtein, jaro_winkler
    return match.extractOne(to_match, choices, match_type=match_type, score_cutoff=0.7)


def unpack_tuple_name(result):
    try:
        return result[0]
    except:
        return np.nan

def unpack_tuple_score(result):
    try:
        return result[1]
    except:
        return np.nan
    
start = time.time()

master_data['fuzzy_brand']= master_data.apply(
    lambda x: match_brand(
        motorcycle_database.brand_db.unique().tolist(),
        x['brand']), 
    axis=1)

master_data['fuzzy_score'] = master_data['fuzzy_brand'].apply(unpack_tuple_score)
master_data['fuzzy_brand'] = master_data['fuzzy_brand'].apply(unpack_tuple_name)

def choices(year, brand, type_name):
   return motorcycle_database[(motorcycle_database.brand_db==brand)&((motorcycle_database.year_db <= 2020+1) | (motorcycle_database.year_db >= 2020-2))][type_name].unique().tolist()

#def choices(brand, type_name):
#    return motorcycle_database[motorcycle_database.brand_db==brand][type_name].unique().tolist()

def match_model(choices, to_match, match_type='levenshtein'): #match_type options include trigram, cosine, levenshtein, jaro_winkler
    return match.extractOne(to_match, choices, match_type=match_type, score_cutoff=0.3)


master_data['fuzzy_result_model']= master_data.apply(lambda x: 
                                              match_model(
                                                  choices(x['bike_year'],x['fuzzy_brand'],'model_db'),
                                                  x['model']), 
                                              axis=1)


master_data['fuzzy_result_submodel']= master_data.apply(lambda x: 
                                                 match_model(
                                                     choices(x['bike_year'],x['fuzzy_brand'],'model_submodel_db'),
                                                     x['model']), 
                                                 axis=1)


master_data['fuzzy_result_submodel_inv']= master_data.apply(lambda x: 
                                                 match_model(
                                                     choices(x['bike_year'],x['fuzzy_brand'],'model_submodel_inv_db'),
                                                     x['model']), 
                                                 axis=1)


master_data['fuzzy_result_model_inv']= master_data.apply(lambda x: 
                                                 match_model(
                                                     choices(x['bike_year'],x['fuzzy_brand'],'model_inv_db'),
                                                     x['model']), 
                                                 axis=1)

master_data['fuzzy_model_score'] = master_data['fuzzy_result_model'].apply(unpack_tuple_score)
master_data['fuzzy_model'] = master_data['fuzzy_result_model'].apply(unpack_tuple_name)

master_data['fuzzy_model_inv_score'] = master_data['fuzzy_result_model_inv'].apply(unpack_tuple_score)
master_data['fuzzy_model_inv'] = master_data['fuzzy_result_model_inv'].apply(unpack_tuple_name)

master_data['fuzzy_submodel_score'] = master_data['fuzzy_result_submodel'].apply(unpack_tuple_score)
master_data['fuzzy_submodel'] = master_data['fuzzy_result_submodel'].apply(unpack_tuple_name)

master_data['fuzzy_submodel_inv_score'] = master_data['fuzzy_result_submodel_inv'].apply(unpack_tuple_score)
master_data['fuzzy_submodel_inv'] = master_data['fuzzy_result_submodel_inv'].apply(unpack_tuple_name)

master_data.drop(columns=['fuzzy_result_model','fuzzy_result_model','fuzzy_result_submodel', 'fuzzy_result_submodel_inv', 'fuzzy_result_model_inv'],inplace=True)

def is_best(fuzzy_model_score, 
            fuzzy_model_inv_score,
            fuzzy_submodel_score, 
            fuzzy_submodel_inv_score):
    
    scores = [float(fuzzy_model_score), float(fuzzy_model_inv_score), float(fuzzy_submodel_score), float(fuzzy_submodel_inv_score)] 
    
    max_score= max(scores)
    max_score_postion = scores.index(max_score)
    
    return max_score_postion

master_data['is_best']= master_data.apply(
    lambda x: is_best(
        x['fuzzy_model_score'],
        x['fuzzy_model_inv_score'],
        x['fuzzy_submodel_score'],
        x['fuzzy_submodel_inv_score']),
    axis=1)

master_data.dropna(subset=['is_best'], inplace=True)

data_model = master_data.copy()[master_data.is_best==0]
data_model_inv = master_data.copy()[master_data.is_best==1]
data_submodel = master_data.copy()[master_data.is_best==2]
data_submodel_inv = master_data.copy()[master_data.is_best==3]

# motorcycle databse
motorcycle_database_model = motorcycle_database.copy()
motorcycle_database_model.drop_duplicates(subset=['brand_db','model_db'],inplace=True)

motorcycle_database_model_inv = motorcycle_database.copy()
motorcycle_database_model_inv.drop_duplicates(subset=['brand_db','model_inv_db'],inplace=True)

motorcycle_database_submodel = motorcycle_database.copy()
motorcycle_database_submodel.drop_duplicates(subset=['brand_db','model_submodel_db'],inplace=True)

motorcycle_database_submodel_inv = motorcycle_database.copy()
motorcycle_database_submodel_inv.drop_duplicates(subset=['brand_db','model_submodel_inv_db'],inplace=True)

data_model = data_model.merge(
    motorcycle_database_model, 
    how='left', 
    left_on=['fuzzy_brand', 'fuzzy_model'], 
    right_on=['brand_db', 'model_db'])

data_model_inv = data_model_inv.merge(
    motorcycle_database_model_inv, 
    how='left', 
    left_on=['fuzzy_brand', 'fuzzy_model_inv'], 
    right_on=['brand_db', 'model_inv_db'])

data_submodel = data_submodel.merge(
    motorcycle_database_submodel, 
    how='left', 
    left_on=['fuzzy_brand', 'fuzzy_submodel'], 
    right_on=['brand_db', 'model_submodel_db'])

data_submodel_inv = data_submodel_inv.merge(
    motorcycle_database_submodel_inv, 
    how='left', 
    left_on=['fuzzy_brand', 'fuzzy_submodel_inv'], 
    right_on=['brand_db', 'model_submodel_inv_db'])

master_data = data_model.append(data_model_inv)
master_data = master_data.append(data_submodel)
master_data = master_data.append(data_submodel_inv)

master_data = master_data[['uniq_id','brand','model','brand_db','model_db','type_db','category_db','bike_year','mileage','price']]

master_data.to_csv('one_page_fuzzy_matched.csv', index=False)

end = time.time()
print(end-start)

master_data

0.24756622314453125


,uniq_id,brand,model,brand_db,model_db,type_db,category_db,bike_year,mileage,price
0,1189664,yamaha,niken,yamaha,niken,motorcycle,street,2018,4100,10590
